In [ ]:
import os
import json
import logging
import sys
from datetime import datetime
from typing import Dict, Any, List, Optional
from dataclasses import dataclass, asdict
from collections import defaultdict
import uuid
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from autogen.agentchat.contrib.capabilities.transform_messages import TransformMessages


In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)

In [ ]:
# Azure OpenAI Configuration
llm_config = {
    "config_list": [
        {
            "model": "gpt-4o",
            "api_key": "",
            "base_url": "",
            "api_type": "azure",
            "api_version": "2024-02-01"
        }
    ],
    "temperature": 0.7,
    "timeout": 120,
    "cache_seed": 42
}

In [ ]:
# Create directories for data storage
os.makedirs("data/conversation_history", exist_ok=True)
os.makedirs("data/progress_data", exist_ok=True)
os.makedirs("data/student_conversations", exist_ok=True)

In [ ]:

# Agent Configuration
class AgentConfig:
    @staticmethod
    def get_tutor_system_message() -> str:
        return """You are an expert Educational Tutor Agent. Explain concepts clearly, adapt to the student's level, provide step-by-step breakdowns, and encourage critical thinking. Offer examples and practice questions. Respond in a supportive, engaging tone."""

    @staticmethod
    def get_student_system_message() -> str:
        return """You are a high-school Student Agent. Actively engage by:
- Attempting to solve problems and sharing your steps (e.g., 'I factored x^2 - 7x + 10 as (x-5)(x-2)').
- Asking questions if unsure (e.g., 'Can you explain the quadratic formula?').
- Reflecting on learning (e.g., 'I get factoring but struggle with graphing').
- Responding to tutor prompts with effort, even if incomplete."""

    @staticmethod
    def get_progress_tracker_system_message() -> str:
        return """You are a Progress Tracker Agent. Analyze student performance from ProgressMemory, generate detailed progress reports, and suggest focus areas. Provide reports in a student-friendly JSON format."""

In [ ]:


# Conversation Memory
class EnhancedConversationMemory:
    def __init__(self, session_id: Optional[str] = None, storage_path: str = "data/conversation_history"):
        self.session_id = session_id or str(uuid.uuid4())
        self.storage_path = storage_path
        self.conversation_history: List[Dict[str, Any]] = []
        os.makedirs(self.storage_path, exist_ok=True)

    def add_message(self, role: str, content: str, metadata: Optional[Dict[str, Any]] = None):
        message = {"role": role, "content": content, "timestamp": datetime.now().isoformat()}
        self.conversation_history.append(message)

    def get_messages(self) -> List[Dict[str, Any]]:
        return self.conversation_history

In [ ]:



# Progress Memory
@dataclass
class LearningProgress:
    subject: str
    topic: str
    skill_level: float
    attempts: int
    successful_attempts: int

class ProgressMemory:
    def __init__(self, student_id: str, storage_path: str = "data/progress_data"):
        self.student_id = student_id
        self.storage_path = storage_path
        self.progress_data: Dict[str, Dict[str, LearningProgress]] = defaultdict(dict)
        os.makedirs(self.storage_path, exist_ok=True)

    def update_progress(self, subject: str, topic: str, performance_score: float, was_successful: bool):
        if subject not in self.progress_data:
            self.progress_data[subject] = {}
        if topic not in self.progress_data[subject]:
            self.progress_data[subject][topic] = LearningProgress(subject, topic, 0.0, 0, 0)
        progress = self.progress_data[subject][topic]
        progress.attempts += 1
        if was_successful:
            progress.successful_attempts += 1
        progress.skill_level = (progress.skill_level * 0.7) + (performance_score * 0.3)
        logger.info(f"Updated progress: {subject}/{topic}, skill={progress.skill_level}")

    def get_progress_report(self) -> Dict[str, Any]:
        return {
            "student_id": self.student_id,
            "subjects": {s: {t: asdict(p) for t, p in topics.items()} for s, topics in self.progress_data.items()}
        }

# Create Agents
def create_educational_agents(student_id: str, session_id: str):
    progress_memory = ProgressMemory(student_id)
    conv_memory = EnhancedConversationMemory(session_id)

    tutor = AssistantAgent(
        name="Educational_Tutor",
        system_message=AgentConfig.get_tutor_system_message(),
        llm_config=llm_config,
        human_input_mode="NEVER",
        max_consecutive_auto_reply=5
    )

    student = UserProxyAgent(
        name="Student_Learner",
        system_message=AgentConfig.get_student_system_message(),
        human_input_mode="NEVER",
        max_consecutive_auto_reply=3,
        code_execution_config=False
    )

    progress_tracker = AssistantAgent(
        name="Progress_Tracker",
        system_message=AgentConfig.get_progress_tracker_system_message(),
        llm_config=llm_config,
        human_input_mode="NEVER"
    )

    def generate_progress_report():
        report = progress_memory.get_progress_report()
        prompt = f"Generate a JSON progress report based on:\n{json.dumps(report, indent=2)}\nInclude summary, strengths, and recommendations."
        response = progress_tracker.generate_reply([{"content": prompt, "role": "user"}])
        json_start = response.find('{')
        json_end = response.rfind('}') + 1
        return json.loads(response[json_start:json_end]) if json_start != -1 and json_end != 0 else {"error": "No report"}

    progress_tracker.register_function({"generate_progress_report": generate_progress_report})

    def custom_speaker_selection(last_speaker, groupchat):
        agents = groupchat.agents
        next_idx = (agents.index(last_speaker) + 1) % len(agents)
        last_message = groupchat.messages[-1]["content"].strip() if groupchat.messages else ""
        if last_speaker == student and not last_message:
            return tutor  # Skip empty student responses
        return agents[next_idx]

    groupchat = GroupChat(
        agents=[student, tutor, progress_tracker],
        messages=[],
        max_round=20,
        speaker_selection_method=custom_speaker_selection
    )

    manager = GroupChatManager(
        groupchat=groupchat,
        llm_config=llm_config,
        name="Education_Manager"
    )

    return {
        "student": student,
        "tutor": tutor,
        "progress_tracker": progress_tracker,
        "manager": manager,
        "progress_memory": progress_memory,
        "conv_memory": conv_memory
    }



In [ ]:

def main():
    print("=== Starting Educational Session ===")
    student_id = "test_student_001"
    session_id = str(uuid.uuid4())
    agents = create_educational_agents(student_id, session_id)

    student = agents["student"]
    manager = agents["manager"]
    progress_tracker = agents["progress_tracker"]
    progress_memory = agents["progress_memory"]
    conv_memory = agents["conv_memory"]

    # Student asks about quadratic equations
    print("\n=== Student Asking About Quadratic Equations ===")
    student.initiate_chat(
        manager,
        message="For x^2 - 7x + 10 = 0, I tried factoring. I got x = 2 and x = 5. Is this correct?"
    )
    progress_memory.update_progress("Mathematics", "Quadratic Equations", 0.8, True)
    conv_memory.add_message("student", "Factored x^2 - 7x + 10", {"subject": "Mathematics"})

    # Tutor responds (handled by GroupChat), then student follows up
    print("\n=== Student Follow-Up ===")
    student.send(
        message="Can you give me another factoring problem to try?",
        recipient=manager
    )

    # Generate progress report
    print("\n=== Generating Progress Report ===")
    response = student.initiate_chat(
        progress_tracker,
        message="Please generate a progress report."
    )
    report = None
    for msg in response.chat_history:
        content = msg.get("content", "")
        if isinstance(content, str) and "summary" in content:
            json_start = content.find('{')
            json_end = content.rfind('}') + 1
            if json_start != -1 and json_end != 0:
                report = json.loads(content[json_start:json_end])
                break
    if report:
        print(json.dumps(report, indent=2))
    else:
        print("Failed to generate progress report.")

    print("\n=== Educational Session Completed ===")



In [ ]:
if __name__ == "__main__":
    os.environ["AUTOGEN_USE_DOCKER"] = "False"
    main()